In [2]:
import torch
import transformers
import trl

print(f"PyTorch version: {torch.__version__}")
print(f"Transformers version: {transformers.__version__}")
print(f"TRL version: {trl.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")

PyTorch version: 2.4.1+cu124
Transformers version: 4.44.2
TRL version: 0.8.6
CUDA available: True
CUDA version: 12.4


In [ ]:
#토큰값 넣기
from huggingface_hub import login

login(
  token="my_api_key",
  add_to_git_credential=True
)


In [4]:
!git config --global credential.helper store

In [5]:
import json 
import torch
from datasets import Dataset, load_dataset
from trl import (setup_chat_format, 
                 DataCollatorForCompletionOnlyLM, 
                 SFTTrainer)
from peft import AutoPeftModelForCausalLM, LoraConfig, PeftConfig 
from transformers import (AutoTokenizer, 
                          AutoModelForCausalLM, 
                          TrainingArguments, 
                          BitsAndBytesConfig, 
                          pipeline, 
                          StoppingCriteria)

model_id = "google/gemma-2-9b-it" 

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    attn_implementation='eager'
    # load_in_8bit=True
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
!wget https://raw.githubusercontent.com/MrBananaHuman/CounselGPT/main/total_kor_multiturn_counsel_bot.jsonl

--2025-08-11 02:28:52--  https://raw.githubusercontent.com/MrBananaHuman/CounselGPT/main/total_kor_multiturn_counsel_bot.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30560672 (29M) [application/octet-stream]
Saving to: ‘total_kor_multiturn_counsel_bot.jsonl.1’

total_kor_multiturn 100%[===================>]  29.14M  58.4MB/s    in 0.5s    

2025-08-11 02:28:54 (58.4 MB/s) - ‘total_kor_multiturn_counsel_bot.jsonl.1’ saved [30560672/30560672]



In [7]:
with open("./total_kor_multiturn_counsel_bot.jsonl") as f:
    original_jsonl_data = [json.loads(line) for line in f ]

In [8]:
len(original_jsonl_data)

8731

In [9]:
original_jsonl_data[1]

[{'speaker': '상담사', 'utterance': '안녕하세요. 어떤 고민으로 오셨나요?'},
 {'speaker': '내담자',
  'utterance': '회사에서 일이 너무 많아서 스트레스 받고 있습니다. 하루종일 업무만 하다가 집에와서도 계속해서 일을 한다는 생각이 들어 매우 힘들어지고 있습니다. 또한, 회사에서 동료들과의 갈등으로 인해 상사와 더 이상 일을 하기 어렵게 되었습니다. 그래서 이제는 회사에서 일을 하기가 싫어지고, 자꾸만 회사에서 일하는 것에 대한 걱정만 하게 됩니다. 이렇게 되면 회사에서 일을 제대로 하지 못하고 지속적으로 피로감과 스트레스가 느껴져서 힘들어져 버리는데, 어떻게 해결할 수 있을까요?'},
 {'speaker': '상담사',
  'utterance': '이해했습니다. 일과 삶의 균형이 깨져서 스트레스가 쌓이시는 것 같아요. 지금 회사에서 경험하고 있는 상황은 일과 삶의 균형이 깨져서 스트레스가 늘어나는 일과 직장인의 대표적인 문제중 하나인 대인관계 문제입니다. 일이 너무 바쁘면 집과 회사 사이에서 적절한 여유가 없어지기 때문에 스트레스가 쌓이고, 새로운 업무를 받았을 때 동료나 상사와 갈등이 생기면서 대인관계 문제가 생길 수 있어요. 제가 알려드릴 수 있는 스트레스 해소 방법을 들어보시겠어요?'},
 {'speaker': '내담자', 'utterance': '네, 좀 알려주세요.'},
 {'speaker': '상담사',
  'utterance': '스트레스를 관리하는 방법에는 다음과 같은 것이 있습니다. 첫째, 몸을 움직이는 것입니다. 운동은 신체적 건강을 챙기는 것 뿐만 아니라 정신적으로도 도움이 됩니다. 둘째, 취미활동을 즐기는 것입니다. 취미활동을 하면 우울증을 예방하고 스트레스를 줄일 수 있어요. 셋째, 정신적인 안정을 취하는 것입니다. 일상적인 규칙적인 생활 패턴을 유지하고 충분한 수면을 취하는 것이 중요합니다. 넷째, 대인관계를 개선하는 것입니다. 문제가 있는 동료나 상사와 직접적인 대화를 해보거나, 간

In [10]:
test_data = original_jsonl_data[0]

In [11]:
speaker_dict = {'내담자':"user" , "상담사":"assistant"}

In [12]:
tmp = []
for m in test_data:
    tmp.append({'role' : speaker_dict[m['speaker']], 'content': m['utterance']})

In [13]:
converted_messages = [{'role' : speaker_dict[m['speaker']], 'content': m['utterance']} for m in test_data]

In [14]:
#assistant로 시작하는 경우 첫번째 메시지 제거 
if converted_messages and converted_messages[0]['role'] == 'assistant':
    converted_messages = converted_messages[1:]
#user로 끝나는 경우 마지막 메시지들 제거 
while converted_messages and converted_messages[-1]['role'] == 'user':
    converted_messages = converted_messages[:-1]

In [15]:
merged = []
current_role = converted_messages[0]['role']
current_contet = converted_messages[0]['content']

In [17]:
#대화의 화자가 연속적일때 병합 
merged = []
current_role = converted_messages[0]['role']
current_content = converted_messages[0]['content']
for message in converted_messages[1:]:
    if message['role'] == current_role:
        current_content = current_content + " " + message['content']
    else:
        merged.append({'role' : current_role, 'content' : current_content})
        current_role = message['role']
        current_content = message['content']
merged.append({'role' : current_role , 'content' : current_content})

In [18]:
converted_messages[-3:]

[{'role': 'assistant',
  'content': '걱정은 당연한 것이죠. 그러나 지나치게 걱정하는 것은 오히려 스트레스를 더 받게 됩니다. 지금 이 곳에 오셔서 이야기를 나누어 봤을 때, 내담자님께서 가진 문제가 그렇게 심각한 것은 아닌 것 같습니다. 하지만, 무엇보다도, 내담자님이 존중하고 사랑하는 마음으로 자신을 바라보며, 나름의 방식으로 살아가는 것이 중요합니다. 무언가를 끼워 맞추어 주려 하지 마시고, 스스로 찾아낼 수 있는 길을 찾아보세요.'},
 {'role': 'user', 'content': '그 말씀대로 할게요.'},
 {'role': 'assistant',
  'content': '그렇게 하셔서 조금이나마 좋아지시길 바라겠습니다. 이후에도 힘든 마음이 계속되면 언제든지 저를 찾아주세요.'}]

In [19]:
#위의 로직을 사용자 함수로 
def preprocess_conversation(messages):
    # speaker를 role로 변환
    converted_messages = [{'role': speaker_dict[m['speaker']], 'content': m['utterance']} for m in messages]
    
    # assistant로 시작하는 경우 첫 메시지 제거
    if converted_messages and converted_messages[0]['role'] == 'assistant':
        converted_messages = converted_messages[1:]
    
    # user로 끝나는 경우 마지막 메시지들 제거
    while converted_messages and converted_messages[-1]['role'] == 'user':
        converted_messages = converted_messages[:-1]
    
    # 연속된 동일 역할의 메시지 병합
    converted_messages = merge_consecutive_messages(converted_messages)
    
    # 대화가 비어있거나 홀수 개의 메시지만 남은 경우 처리
    if not converted_messages or len(converted_messages) % 2 != 0:
        return []
    
    return converted_messages

#대화가 연속된 동일 역할의 메시지 병합 함수
def merge_consecutive_messages(messages):
    if not messages:
        return []
    
    merged = []
    current_role = messages[0]['role']
    current_content = messages[0]['content']
    
    for message in messages[1:]:
        if message['role'] == current_role:
            current_content += " " + message['content']
        else:
            merged.append({'role': current_role, 'content': current_content})
            current_role = message['role']
            current_content = message['content']
    
    merged.append({'role': current_role, 'content': current_content})
    return merged

#위에서 만든 사용자 함수를 사용하여 전체 데이터에 적용할 함수 
def transform_to_new_format(original_data):
    transformed_data = []
    for conversation in original_data:
        processed_conversation = preprocess_conversation(conversation)
        if processed_conversation:
            transformed_data.append(processed_conversation)
    return transformed_data

In [20]:
result = transform_to_new_format(original_data=original_jsonl_data)

In [21]:
result 

[[{'role': 'user', 'content': '내가 약간 중2병 같은 걸 증상을 보이고 있어요.'},
  {'role': 'assistant', 'content': '중2병 증상이라니, 어떤 증상이신 건가요?'},
  {'role': 'user',
   'content': '그러니까 공부하기 싫어하고, 공격적이고, 좀 무례하게 말하고 싶은 게 많아져서 그런 거예요.'},
  {'role': 'assistant',
   'content': '그런 증상이 있으니까 힘드시겠죠. 중2병 같은 것이라고 생각하시는 이유는 무엇인가요?'},
  {'role': 'user', 'content': '막 공부 안하고 이것저것 들먹이고 하고 싶은 게 너무 많아서 그런 거 같아요.'},
  {'role': 'assistant',
   'content': '그런 것도 어쩔 수 없이 찾아오는 시기가 있으니까 무리하지 않도록 해야겠죠. 대학교를 가면서 나아질 것 같았는데, 오히려 더 심해진 것 같다고 하셨죠. 그 원인이 무엇인가요?'},
  {'role': 'user', 'content': '그걸 제가 잘 몰라서 그런 것 같아요. 그냥 더 심해졌다고 느꼈어요.'},
  {'role': 'assistant', 'content': '대학교 생활이 신나고 재밌으신 건 어떤 점이 있나요?'},
  {'role': 'user',
   'content': '학과가 정말 좋아서 즐겁게 수업을 듣고 있어요. 학우들도 좋고 괜찮은 친구들도 많이 만나서 그런 것 같아요.'},
  {'role': 'assistant',
   'content': '즐거운 일도 많이 있으면서 고민거리도 있는 것 같군요. 가사나 소설을 쓰시면서 마음을 풀기도 하신다고 하셨는데, 언제부터 그 습관이 생겨난 건가요?'},
  {'role': 'user',
   'content': '좋은 질문이에요. 좀 자세히 말씀드릴게요. 학교에서 어려운 일이 있었는데, 그 때부터 가사나 소설 같은 것들을 쓰면서 마음을 풀게 되었어요. 

In [22]:
with open("./train_dataset.jsonl" , "w", encoding='utf-8') as f:
    for conversation in result:
        json_obj = {'messages' : conversation}
        json.dump(json_obj, f, ensure_ascii=False)
        f.write("\n")

In [23]:
!tail -n 1 train_dataset.jsonl

{"messages": [{"role": "user", "content": "네, 처음입니다."}, {"role": "assistant", "content": "그렇다면 간단히 자기 소개와 함께 어떤 문제 때문에 상담을 받으려고 하는지 말씀해주시겠어요?"}, {"role": "user", "content": "요즘에 일이 너무 바쁘고 정신적으로 지쳐서 약간 힘들어요. 요즘 일이 더욱 바쁘게 돌아가고 있어서 정신적으로 조금 지쳐있어요. 어떻게 조금이나마 안정적으로 지낼 수 있을까요?"}, {"role": "assistant", "content": "고민하시는 상황이 적지 않네요. 혹시 이 문제 때문에 무슨 불편함이나 고통을 느끼시나요?"}, {"role": "user", "content": "매일 똑같은 일을 반복하고, 한계를 느끼게 되면서 힘들어지기도 하고요. 뭐가 이렇게 억울한지 모르겠어요."}, {"role": "assistant", "content": "이 문제를 어떻게 해결하시려고 하시나요?"}, {"role": "user", "content": "그게 문제예요. 해결책을 찾기가 너무 어려워서 상담을 찾게 되었어요."}, {"role": "assistant", "content": "일이 바쁘면서도 여러가지 불안함을 느끼시는 것 같아요. 어떤 부분에서 어려움을 느끼시는지 좀 더 자세히 이야기해주시겠어요?"}, {"role": "user", "content": "뭔가에 집중하다보면, 동시에 다른 일들도 계속 생기는 느낌이 들어서 힘들어요."}, {"role": "assistant", "content": "일의 분량이 많아지다보면 집중력이 떨어지는 느낌이 드시는군요. 그렇다면 더 효과적인 일의 처리법이 있을 수도 있겠죠? 예를 들어, 한가지 일을 마치고 다음 일을 시작하기 전에 짧은 휴식을 취하는 것이나, 한 번에 모든 일을 처리하려는 것보다 하나씩 순차적으로 해나가는 것이 좋을 수도 있어요."}, {"role": "user", "content": "그건 괜

In [24]:
dataset = load_dataset("json", data_files="./train_dataset.jsonl", split="train")
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['messages'],
    num_rows: 8731
})

In [25]:
peft_config = LoraConfig(
        lora_alpha=128,
        lora_dropout=0.05,
        r=256,
        bias="none",
        target_modules=[
            "q_proj",
            "up_proj",
            "o_proj",
            "k_proj",
            "down_proj",
            "gate_proj",
            "v_proj"], #부분적인 파인 튜닝이 가능해짐.
        task_type="CAUSAL_LM",
)

In [26]:
args = TrainingArguments(
    output_dir="./model_output", 
    num_train_epochs=1,          
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,  
    optim="adamw_torch_fused",    
    logging_steps=100,            
    save_strategy="epoch",        
    learning_rate=2e-4,           
    bf16=True,                    
    tf32=True,                    
    max_grad_norm=0.3,            
    warmup_ratio=0.03,            
    lr_scheduler_type="constant", 
    push_to_hub=False,             
    #report_to="wandb",            
)

In [27]:
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset,
    max_seq_length=512,
    peft_config=peft_config,
    tokenizer=tokenizer,
    packing=True,
)

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [29]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

  ········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 0224seorin (0224seorin-free) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
100,1.393400
200,1.309200
300,1.301900
400,1.288700
500,1.280000
600,1.268400
700,1.277300
800,1.273200
900,1.264700
1000,1.270900


TrainOutput(global_step=1725, training_loss=1.2742359835859658, metrics={'train_runtime': 8963.8803, 'train_samples_per_second': 1.54, 'train_steps_per_second': 0.192, 'total_flos': 3.895332015439872e+17, 'train_loss': 1.2742359835859658, 'epoch': 1.0})